# Projet : Consommation et production électrique en France

## Présentation Du Dataset



**Informations issues du site (allez voir vous même également) :**

Ce jeu de données, rafraîchi une fois par jour, présente les données régionales de janvier 2013 à 2023. Elles sont issues de l'application éCO2mix. Elles sont élaborées à partir des comptages et complétées par des forfaits. 

Vous y trouverez au pas quart d'heure :

- Les prévisions de consommation établies la veille (J-1) et celles réactualisées le jour même (J).

Vous y trouverez au pas demi-heure :

- La consommation réalisée.
- La production selon les différentes filières composant le mix énergétique.
- La consommation des pompes dans les Stations de Transfert d'Energie par Pompage (STEP).
- Les échanges physiques aux frontières.
- Une estimation des émissions de carbone générées par la production d'électricité en France.
- Les échanges commerciaux aux frontières.
- Le découpage des filières par technologie du mix de production (débute en 2013).

**Modalités de l'étude :** 

- Vous travaillerez en groupe de 2 à 3.    
   
- Une présentation de votre travail sur une question sera effectuée en fin du projet. 
   
**Les objectifs de cette étude sont multiples :** 

- Apprendre à charger et manipuler des données réelles complexes avec Pandas. 

- Manipuler des séries temporelles. 

- Analyser des données pour répondre à une question exploratoire. 

- Présenter et vulgariser votre recherche exploratoire. 

**Notes sur les données RTE** :

- Elles proviennent du (génial) site éCO2mix et sont disponibles pour tout le monde (opendata) : https://www.rte-france.com/eco2mix 
- Données agrégées au niveau national : https://opendata.reseaux-energies.fr/explore/dataset/eco2mix-national-cons-def/information/?disjunctive.nature 
- Données agrégées au niveau régional : https://opendata.reseaux-energies.fr/explore/dataset/eco2mix-regional-cons-def/information/?disjunctive.libelle_region&disjunctive.nature

# Chargement et préparation des données

&#x1F4A5; **To Do** 

- Charger les données nationales.
 
- Regarder les colonnes, sélectionner les données intéressantes.
 
- Les types inférés semblent-ils corrects ? Corriger si nécessaire (datetime, object, int, float, etc.)

- Choisir des noms de colonnes plus faciles à manipuler (espaces, accents...). <br/>Ressource : https://www.dataschool.io/pandas-dot-notation-vs-brackets/

- Choisir un index adéquat pour votre dataframe. Les lignes ont un comportement étrange, choisissez les lignes qui vous arrangent.

- Quelle période temporelle couvrent les données ?

- Faîtes un choix par rapport aux valeurs manquantes. Les garder sous forme de NaN ou une autre valeur ?

- Simplifiez le DataFrame : regardez vos colonnnes, cherchez à les comprendre et rassemblez celles que vous pouvez rassembler. 

- Sauvegardez vos données sous format **pkl** pour ne pas devoir refaire les pré-traitements à chaque fois que vous redémarrez le kernel.  


&#x1F4A5; **Ressources**

- La doc de pandas.

- Voici quelques fonctions en vrac dont vous aurez besoin (read_csv,  info,  drop, to_datetime, astype, nunique, set_index)

In [1]:
import pandas as pd

#### Chargement des données

In [2]:
df_consommation_regional = pd.read_parquet('./data_eco2/eco2mix-regional-cons-def.parquet')
df_consommation_national = pd.read_parquet('./data_eco2/eco2mix-national-cons-def.parquet')

- Regarder les colonnes, sélectionner les données intéressantes.

In [3]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_info_columns', 1000)
pd.set_option('display.width', 1000)
df_consommation_national.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388704 entries, 0 to 388703
Data columns (total 37 columns):
 #   Column                       Non-Null Count   Dtype                        
---  ------                       --------------   -----                        
 0   perimetre                    388704 non-null  object                       
 1   nature                       388704 non-null  object                       
 2   date                         388704 non-null  object                       
 3   heure                        388704 non-null  object                       
 4   date_heure                   388704 non-null  datetime64[ms, Europe/Berlin]
 5   consommation                 194352 non-null  float64                      
 6   prevision_j1                 388704 non-null  int64                        
 7   prevision_j                  388704 non-null  int64                        
 8   fioul                        194352 non-null  float64                     

In [4]:
print(df_consommation_national.perimetre.unique())
print(df_consommation_national.nature.unique())

['France']
['Données définitives' 'Données consolidées']


# Traitement des données

## Selection des colonnes

Dans un premier temps , on va etudier les données de consommation et de production général on va donc supprimer des colonnes de details production (fioul_cogen	...)

De la meme manière on va garder seulement le colonne 'ech_physiques' et supprimer le datail par pays

Nous avons aussi des données qui donne une information en double on va donc garder :
date_heure et suppression de date, heure

On va aussi supprimer les données de prévisions qui ne nous interesse pas pour le moment

Enfin perimetre n'a qu'une valeur , on peut supprimer la colonne

In [5]:
# we keep the columns : fioul, charbon, gaz, nucleaire,	eolien,	solaire, hydraulique, pompage, bioenergies,
df_national_reduit= df_consommation_national[['nature','date_heure','consommation', 'fioul', 'charbon', 'gaz', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies','ech_physiques','taux_co2']].copy()

## Verification du datetime

Nous avons des données temporelles , nous aller verifier que les valeurs de date_heure sont sur le meme fuseau horaire , sinon on convertira tout en UTC +2

In [13]:
df_national_reduit['date_heure'].apply(lambda x: x.utcoffset().total_seconds() / 3600).unique()

array([1., 2.])

On verifie si des moments sont dupliqué

In [ ]:
df_national_reduit.sort_values(['date_heure']).head(5)
df_national_reduit['date_heure_gmt2'] = df_national_reduit['date_heure'].dt.tz_convert('Etc/GMT-2')
df_national_reduit[df_national_reduit.duplicated(subset=['date_heure_gmt2'],keep=False)].sort_values(['date_heure_gmt2'])

,nature,date_heure,consommation,fioul,charbon,gaz,nucleaire,eolien,solaire,hydraulique,pompage,bioenergies,ech_physiques,taux_co2,date_heure_gmt2
932,Données définitives,2012-03-25 03:00:00+02:00,46435.0,480.0,25.0,3561.0,42185.0,1225.0,0.0,3887.0,-476.0,753.0,-5206.0,42.0,2012-03-25 03:00:00+02:00
933,Données définitives,2012-03-25 03:00:00+02:00,46435.0,480.0,25.0,3561.0,42185.0,1225.0,0.0,3887.0,-476.0,753.0,-5206.0,42.0,2012-03-25 03:00:00+02:00
30341,Données définitives,2012-03-25 03:15:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-25 03:15:00+02:00
30342,Données définitives,2012-03-25 03:15:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-25 03:15:00+02:00
367928,Données définitives,2012-03-25 03:30:00+02:00,46383.0,479.0,25.0,3565.0,42642.0,1183.0,0.0,3918.0,-647.0,738.0,-5520.0,42.0,2012-03-25 03:30:00+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124943,Données définitives,2022-03-27 03:15:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-27 03:15:00+02:00
158741,Données définitives,2022-03-27 03:30:00+02:00,46073.0,98.0,13.0,5638.0,29072.0,6566.0,1.0,5783.0,-116.0,1243.0,-2145.0,52.0,2022-03-27 03:30:00+02:00
124944,Données définitives,2022-03-27 03:30:00+02:00,46073.0,98.0,13.0,5638.0,29072.0,6566.0,1.0,5783.0,-116.0,1243.0,-2145.0,52.0,2022-03-27 03:30:00+02:00
124942,Données définitives,2022-03-27 03:45:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-27 03:45:00+02:00


In [15]:
index1=list(df_national_reduit[df_national_reduit.duplicated(keep=False)].sort_values(['date_heure_gmt2']).index)
index2=list(df_national_reduit[df_national_reduit.duplicated(subset=['date_heure_gmt2'],keep=False)].sort_values(['date_heure_gmt2']).index)
df_national_reduit.loc[list(set(index2) - set(index1))].sort_values(['date_heure_gmt2'])

,nature,date_heure,consommation,fioul,charbon,gaz,nucleaire,eolien,solaire,hydraulique,pompage,bioenergies,ech_physiques,taux_co2,date_heure_gmt2
182536,Données définitives,2012-03-25 03:30:00+02:00,47196.0,480.0,22.0,3561.0,43495.0,1271.0,0.0,3259.0,-1137.0,754.0,-4508.0,42.0,2012-03-25 03:30:00+02:00
367928,Données définitives,2012-03-25 03:30:00+02:00,46383.0,479.0,25.0,3565.0,42642.0,1183.0,0.0,3918.0,-647.0,738.0,-5520.0,42.0,2012-03-25 03:30:00+02:00
216135,Données définitives,2013-03-31 03:30:00+02:00,62694.0,482.0,3448.0,2778.0,46075.0,1925.0,0.0,8933.0,-984.0,727.0,-690.0,83.0,2013-03-31 03:30:00+02:00
63540,Données définitives,2013-03-31 03:30:00+02:00,62765.0,481.0,3429.0,2808.0,45807.0,1946.0,0.0,9478.0,-769.0,727.0,-1144.0,83.0,2013-03-31 03:30:00+02:00
264970,Données définitives,2014-03-30 03:30:00+02:00,47855.0,280.0,-11.0,1968.0,43783.0,2205.0,-1.0,8193.0,-6.0,965.0,-9521.0,31.0,2014-03-30 03:30:00+02:00
43628,Données définitives,2014-03-30 03:30:00+02:00,48737.0,280.0,-11.0,1959.0,44135.0,2318.0,-1.0,6789.0,-996.0,952.0,-6688.0,32.0,2014-03-30 03:30:00+02:00
72028,Données définitives,2015-03-29 03:30:00+02:00,52852.0,292.0,-4.0,2041.0,45675.0,6579.0,0.0,4862.0,-2290.0,980.0,-5282.0,32.0,2015-03-29 03:30:00+02:00
232567,Données définitives,2015-03-29 03:30:00+02:00,52100.0,292.0,-5.0,2047.0,45756.0,6439.0,0.0,4714.0,-2535.0,989.0,-5597.0,32.0,2015-03-29 03:30:00+02:00
84953,Données définitives,2016-03-27 03:30:00+02:00,52243.0,156.0,12.0,2067.0,41930.0,5183.0,-1.0,4430.0,-1740.0,1072.0,-867.0,36.0,2016-03-27 03:30:00+02:00
171802,Données définitives,2016-03-27 03:30:00+02:00,51335.0,159.0,13.0,2068.0,40610.0,4903.0,-1.0,3866.0,-1665.0,1073.0,310.0,37.0,2016-03-27 03:30:00+02:00


Pour les heures qui sont dupliqué on remarque que les données sont égales ou proches , je fais donc le choix de ne garder qu'une ligne à chaque fois.
Puis on pourra mettre date_heure en index

In [ ]:

df_national_reduit.drop_duplicates(subset=['date_heure_gmt2'], keep='first', inplace=True)
df_national_reduit = df_national_reduit.set_index('date_heure_gmt2')
df_national_reduit = df_national_reduit.drop(columns=['date_heure'])
df_national_reduit = df_national_reduit.sort_index()



In [17]:
df_national_reduit

,nature,consommation,fioul,charbon,gaz,nucleaire,eolien,solaire,hydraulique,pompage,bioenergies,ech_physiques,taux_co2
date_heure_gmt2,,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,Données définitives,58315.0,492.0,25.0,3816.0,52697.0,3588.0,0.0,7922.0,-1139.0,719.0,-9806.0,33.0
2012-01-01 00:15:00+01:00,Données définitives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 00:30:00+01:00,Données définitives,58315.0,492.0,25.0,3816.0,52697.0,3588.0,0.0,7922.0,-1139.0,719.0,-9806.0,33.0
2012-01-01 00:45:00+01:00,Données définitives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 01:00:00+01:00,Données définitives,56231.0,492.0,25.0,3834.0,51747.0,3536.0,0.0,7598.0,-1730.0,721.0,-9993.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 22:45:00+01:00,Données consolidées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 23:00:00+01:00,Données consolidées,64893.0,84.0,22.0,5497.0,42602.0,6027.0,0.0,8720.0,-20.0,1115.0,944.0,41.0
2023-01-31 23:15:00+01:00,Données consolidées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lorsque que les minutes se terminent par 15 ou 45 nous n'avons pas de valeurs (voir doc) , on va donc supprimer ces colonnes

In [18]:
#Delete row if minute end by 15 ou 45
df_national_reduit = df_national_reduit[~df_national_reduit.index.minute.isin([15, 45])].copy()
df_national_reduit 

,nature,consommation,fioul,charbon,gaz,nucleaire,eolien,solaire,hydraulique,pompage,bioenergies,ech_physiques,taux_co2
date_heure_gmt2,,,,,,,,,,,,,
2012-01-01 00:00:00+01:00,Données définitives,58315.0,492.0,25.0,3816.0,52697.0,3588.0,0.0,7922.0,-1139.0,719.0,-9806.0,33.0
2012-01-01 00:30:00+01:00,Données définitives,58315.0,492.0,25.0,3816.0,52697.0,3588.0,0.0,7922.0,-1139.0,719.0,-9806.0,33.0
2012-01-01 01:00:00+01:00,Données définitives,56231.0,492.0,25.0,3834.0,51747.0,3536.0,0.0,7598.0,-1730.0,721.0,-9993.0,34.0
2012-01-01 01:30:00+01:00,Données définitives,56075.0,491.0,25.0,3832.0,51950.0,3526.0,0.0,7299.0,-2134.0,722.0,-9636.0,35.0
2012-01-01 02:00:00+01:00,Données définitives,55532.0,492.0,25.0,3839.0,51625.0,3535.0,0.0,7159.0,-2449.0,719.0,-9412.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 21:30:00+01:00,Données consolidées,64603.0,83.0,19.0,5395.0,42577.0,5961.0,0.0,8786.0,-15.0,1104.0,797.0,40.0
2023-01-31 22:00:00+01:00,Données consolidées,63276.0,83.0,20.0,5134.0,42437.0,5886.0,0.0,7892.0,-15.0,1101.0,844.0,40.0
2023-01-31 22:30:00+01:00,Données consolidées,63542.0,83.0,22.0,5404.0,42568.0,5923.0,0.0,7951.0,-20.0,1114.0,601.0,41.0


## Gestion des types

In [ ]:
df_national_reduit.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 194330 entries, 2012-01-01 01:00:00+02:00 to 2023-02-01 00:30:00+02:00
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   nature         194330 non-null  object 
 1   consommation   194330 non-null  float64
 2   fioul          194330 non-null  float64
 3   charbon        194330 non-null  float64
 4   gaz            194330 non-null  float64
 5   nucleaire      194330 non-null  float64
 6   eolien         194330 non-null  float64
 7   solaire        194330 non-null  float64
 8   hydraulique    194330 non-null  float64
 9   pompage        194330 non-null  float64
 10  bioenergies    194330 non-null  float64
 11  ech_physiques  194330 non-null  float64
 12  taux_co2       194330 non-null  float64
dtypes: float64(12), object(1)
memory usage: 20.8+ MB


Nos types sont float mais la précisions apporté par les valeurs décimal est négligable , on convertit donc nos float en int32

In [ ]:
# Convert float columns to int32 after handling NaN values
for col in ['fioul', 'charbon', 'gaz', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies']:
    df_national_reduit[col] = df_national_reduit[col].fillna(0).astype('int32')

## Enregistrement

In [ ]:
#Save data in pkl format
df_national_reduit.to_pickle('./data_eco2/eco2mix-national-cons-def-reduit.pkl')